<a href="https://colab.research.google.com/github/FreddyGG/TP_BIG_DATA/blob/main/Trabajo_Pr%C3%A1ctico_de_Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **COMPONENTE 1**

Como paso inicial del proyecto de big data, el alumno escribir un script que permita obtener una
lista de las últimas menciones de un handle o hashtag en Twitter a través de su API. Este
programa en Python deberá luego guardar los tweets en un archivo .csv para su posterior
análisis.
El archivo de salida deberá contener mínimamente: Texto del tweet, Autor, Timestamp, Número de Favs, Número de RTs

In [1]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [2]:
!pip install langdetect

     |████████████████████████████████| 983kB 20.0MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-cp37-none-any.whl size=993242 sha256=250d41d303e1d43fb53f77309f99a1dcfb151b0b4c15151efb790bff64882368
  Stored in directory: /root/.cache/pip/wheels/7e/18/13/038c34057808931c7ddc6c92d3aa015cf1a498df5a70268996
Successfully built langdetect


In [3]:
#importamos librerias
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import SnowballStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('spanish'))
from sklearn.feature_extraction.text import CountVectorizer
from langdetect import detect

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
# Twitter Api Credentials
consumerKey = 'ogYNVLjPL8ygj4PO6vIyYVxyP'#log["key"][0]
consumerSecret = 'fS8uxKMb2hZ4GOpey1Y6AsbWDeQCIA9srJRetzCo9cAMAM6AbZ'#log["key"][1]
accessToken = '175984347-syxFReYiggzkhkr0gz5bKX4Asi5S91eNXKU0iIbV'#log["key"][2]
accessTokenSecret = '1LhUQr7jCEDxZ0BBQSs9TK2VaXiZq08f2pKMMoBPq1GkJ'#log["key"][3]

In [5]:
#creamos un objeto de autenticacion
authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret) 
    
# Seteamos el access token y access token secret
authenticate.set_access_token(accessToken, accessTokenSecret) 

#Creamos el objeto API mientras parseamos la informacion en auth  
api = tweepy.API(authenticate, wait_on_rate_limit = True)

In [18]:
#Pedimos que ingrese los tweets a buscar
print('Analizar Sentimientos de Tweets
')
keyword = input("Ingrese el hastag o keyword que desea buscar: ")
nroTweet = int(input("Ingrese cuantos tweets desea buscar: "))

list = tweepy.Cursor(api.search, q=keyword ,tweet_mode='extended', lang='es').items(nroTweet)

Analizar Sentimientos de Tweets
Ingrese el hastag o keyword que desea buscar: Paraguay
Ingrese cuantos tweets desea buscar: 100


In [19]:
#Almacenamos los tweets en una lista
tweet_list = []
for tweet in list:
    text = tweet._json["full_text"]
    #print(text)
    favourite_count = tweet.favorite_count
    retweet_count = tweet.retweet_count
    created_at = tweet.created_at
    author = tweet.author.name
    
    line = {'text' : text, 'author' : author, 'favourite_count' : favourite_count, 'retweet_count' : retweet_count, 'created_at' : created_at}
    tweet_list.append(line)

In [20]:
#Convertimos la lista a un DataFrame
dfTweets = pd.DataFrame(tweet_list)

#Seteamos la funcion para limpiar los tweets
def cleanTxt(text):
 text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
 text = re.sub('#', '', text) # Removing '#' hash tag
 text = re.sub('RT[\s]+', '', text) # Removing RT
 text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
 
 return text

#Limpiamos los tweets y renombramos las columnas
dfTweets = dfTweets.rename(columns={'text': 'Tweets', 'author': 'Author', 'favourite_count': 'Favourites', 'retweet_count': 'Retweets', 'created_at': 'Date'})
dfTweets['Tweets'] = dfTweets['Tweets'].apply(cleanTxt)

#Creamos el archivo csv
dfTweets.to_csv('tweets.csv')

In [21]:
dfTweets

,Tweets,Author,Favourites,Retweets,Date
0,: Cifra que alarma 💧 | El río Paraná continúa ...,ramon carcilio,0,5,2021-06-23 17:20:18
1,Una universidad de Paraguay aceptará pagos en ...,WAYDN,0,0,2021-06-23 17:20:14
2,_starship: [🪐] FNS | 🐣\n\n(una selca de) Días ...,เบลเซบับวันนี้ไม่รู้จะเป็นอะไรนึกไม่ออก,0,11,2021-06-23 17:20:11
3,★ ★ Una universidad de Paraguay aceptará pag...,MUNDO BITCOIN,0,0,2021-06-23 17:20:09
4,"SANCIONADO I Proyecto de Ley, “QUE DECLARA EST...",Radio Cámara Py,0,0,2021-06-23 17:20:07
...,...,...,...,...,...
95,"20: Se perdió Titan! Zona chango mas, entre Ch...",Marimar,0,14,2021-06-23 17:09:01
96,: Colecta de 3.500.000 para salvar el ojito de...,zuni,0,470,2021-06-23 17:08:59
97,"""Anthony, toca Riveros... Si bien hubo situaci...",¹⁰,1,0,2021-06-23 17:08:42
98,"10: La gente no renuncia ""porque no quiere tra...",negrita,0,162,2021-06-23 17:08:37


# **COMPONENTE 2**

El alumno deberá crear un script en Python que calcule el sentimiento de cada tweet y
almacene los resultados en un archivo .csv. Además, se deberá calcular el sentimiento
promedio relacionado a las menciones.

In [22]:
#creamos un data frame con unicamente los tweets
tweets = pd.DataFrame(dfTweets['Tweets'])

In [23]:
tweets.head()

,Tweets
0,: Cifra que alarma 💧 | El río Paraná continúa ...
1,Una universidad de Paraguay aceptará pagos en ...
2,_starship: [🪐] FNS | 🐣\n\n(una selca de) Días ...
3,★ ★ Una universidad de Paraguay aceptará pag...
4,"SANCIONADO I Proyecto de Ley, “QUE DECLARA EST..."


In [24]:
# crear la funcion para obtener la subjectividad
def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity

# Crear la funcion para obtener la polaridad
def getPolarity(text):
   return  TextBlob(text).sentiment.polarity


# Crear dos nuevas columnas 'Subjectividad' & 'Polaridad'
tweets['Subjectividad'] = tweets['Tweets'].apply(getSubjectivity)
tweets['Polaridad'] = tweets['Tweets'].apply(getPolarity)

# Show the new dataframe with columns 'Subjectivity' & 'Polarity'
tweets

,Tweets,Subjectividad,Polaridad
0,: Cifra que alarma 💧 | El río Paraná continúa ...,0.0,0.00
1,Una universidad de Paraguay aceptará pagos en ...,0.0,0.00
2,_starship: [🪐] FNS | 🐣\n\n(una selca de) Días ...,1.0,-0.75
3,★ ★ Una universidad de Paraguay aceptará pag...,0.0,0.00
4,"SANCIONADO I Proyecto de Ley, “QUE DECLARA EST...",0.0,0.00
...,...,...,...
95,"20: Se perdió Titan! Zona chango mas, entre Ch...",0.0,0.00
96,: Colecta de 3.500.000 para salvar el ojito de...,0.0,0.00
97,"""Anthony, toca Riveros... Si bien hubo situaci...",1.0,0.00
98,"10: La gente no renuncia ""porque no quiere tra...",0.2,-0.10


In [25]:
# Crear una funcion para calcular negativo (-1), neutral (0) and positivo (+1) 
def getAnalysis(score):
  if score < 0:
    return 'Negativo'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positivo'

tweets['Analisis'] = tweets['Polaridad'].apply(getAnalysis)
# Show the dataframe
tweets

,Tweets,Subjectividad,Polaridad,Analisis
0,: Cifra que alarma 💧 | El río Paraná continúa ...,0.0,0.00,Neutral
1,Una universidad de Paraguay aceptará pagos en ...,0.0,0.00,Neutral
2,_starship: [🪐] FNS | 🐣\n\n(una selca de) Días ...,1.0,-0.75,Negativo
3,★ ★ Una universidad de Paraguay aceptará pag...,0.0,0.00,Neutral
4,"SANCIONADO I Proyecto de Ley, “QUE DECLARA EST...",0.0,0.00,Neutral
...,...,...,...,...
95,"20: Se perdió Titan! Zona chango mas, entre Ch...",0.0,0.00,Neutral
96,: Colecta de 3.500.000 para salvar el ojito de...,0.0,0.00,Neutral
97,"""Anthony, toca Riveros... Si bien hubo situaci...",1.0,0.00,Neutral
98,"10: La gente no renuncia ""porque no quiere tra...",0.2,-0.10,Negativo


In [26]:
#Porcentaje de Tweets Positivos
ntweets = tweets[tweets.Analisis == 'Positivo']
ntweets = ntweets['Tweets']
ntweets

round( (ntweets.shape[0] / tweets.shape[0]) * 100, 1)

9.0

In [27]:
#Porcentaje de Tweets Negativos
# Print the percentage of negative tweets
ntweets = tweets[tweets.Analisis == 'Negativo']
ntweets = ntweets['Tweets']
ntweets

round( (ntweets.shape[0] / tweets.shape[0]) * 100, 1)

8.0

In [28]:
#Porcentaje de Tweets Neutros
ntweets = tweets[tweets.Analisis == 'Neutral']
ntweets = ntweets['Tweets']
ntweets

round( (ntweets.shape[0] / tweets.shape[0]) * 100, 1)

83.0